# Project 3.  Due October 31

In [2]:
import sys
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy.linalg as LA

### The attached file, 'polluteddata.csv', contains the following data.
### We took intraday price values for an asset; this time series was then split into half-hour intervals for a total of 1709 intervals

In [3]:
data = pd.read_csv('polluteddata.csv',header=None)
polluteddata = data.to_numpy()

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,0.0,-3.00,-8.26,-10.55,-8.56,-18.08,-12.67,-14.29,-16.08,...,-27.58,-31.51,-27.53,-25.24,-25.32,-24.44,-28.04,-30.94,-29.57,-27.56
1,1.0,0.0,0.12,-0.41,-2.56,0.15,-1.65,-5.14,-5.09,-8.06,...,-6.65,-5.43,-6.13,-3.42,-4.48,-4.85,-4.47,-4.68,-2.10,-0.28
2,2.0,0.0,-2.80,-3.46,-5.80,-3.86,-2.60,-5.30,-4.81,-3.30,...,-12.74,-13.02,-15.99,-14.15,-14.92,-16.80,-16.88,-19.53,-25.30,-19.60
3,3.0,0.0,-1.95,-9.58,-13.68,-13.64,-18.36,-22.33,-15.70,-16.69,...,-19.50,-17.76,-18.52,-20.12,-17.35,-14.18,-15.57,-15.42,-19.58,-17.68
4,4.0,0.0,-1.11,1.62,-0.34,-5.76,-2.53,-0.56,1.45,0.62,...,0.49,-3.04,-0.82,-8.32,-3.87,-10.33,-11.46,-8.94,-16.81,-16.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,1704.0,0.0,0.38,-0.73,-3.09,-2.31,-2.46,-7.22,-5.85,-4.63,...,-23.96,-19.09,-17.55,-23.27,-20.66,-18.36,-18.05,-16.96,-21.37,-25.62
1705,1705.0,0.0,-3.21,-9.32,-2.90,-7.39,-8.25,1.42,1.40,2.22,...,4.42,4.05,1.48,-7.24,-7.92,-15.95,-18.73,-14.73,-12.64,-12.91
1706,1706.0,0.0,-0.83,1.65,2.19,1.62,0.45,0.63,4.18,4.02,...,18.51,18.66,16.63,14.55,16.01,13.66,15.52,15.37,16.70,12.52
1707,1707.0,0.0,0.34,-0.90,-2.51,-2.15,-4.72,-4.57,-7.59,-5.22,...,0.95,3.28,-0.92,0.28,-2.47,-7.22,-9.03,-10.33,-12.30,-16.24


In [5]:
data.shape

(1709, 31)

### There are 1709 lines in this file, one for each half-hour interval.  Each line begins with the count of the corresponding half-hour

In [6]:
for t in range(3): #len(polluteddata[0,:])):
    print('half-hour',t, 'data:',polluteddata[t,:])

half-hour 0 data: [  0.     0.    -3.    -8.26 -10.55  -8.56 -18.08 -12.67 -14.29 -16.08
 -20.04 -15.81 -14.56 -17.34 -20.14 -15.96 -14.3  -15.82 -15.07 -12.39
 -20.65 -27.58 -31.51 -27.53 -25.24 -25.32 -24.44 -28.04 -30.94 -29.57
 -27.56]
half-hour 1 data: [  1.     0.     0.12  -0.41  -2.56   0.15  -1.65  -5.14  -5.09  -8.06
  -9.05 -14.9  -18.27 -16.18 -15.42 -13.8  -12.89 -12.87 -10.18 -12.7
  -9.21  -6.65  -5.43  -6.13  -3.42  -4.48  -4.85  -4.47  -4.68  -2.1
  -0.28]
half-hour 2 data: [  2.     0.    -2.8   -3.46  -5.8   -3.86  -2.6   -5.3   -4.81  -3.3
  -0.28   1.33   0.68   2.3    1.65  -9.32 -11.74 -11.49 -19.37 -11.8
 -10.87 -12.74 -13.02 -15.99 -14.15 -14.92 -16.8  -16.88 -19.53 -25.3
 -19.6 ]


### The data was constructed as follows for each half hour interval $t$.  Suppose that $p(t, k)$ denotes the price of the asset in minute $k$ of half-hour $t$.  Here $k = 0,1,\ldots, 29$.  
### The data in position $k+1$ of line $t$ of the file equals $p(t,k) - p(t,0)$, i.e. the change in price between minute $k$ and minute $0$.

### EXCEPT that:
### The data was <font color = "red">polluted</font> in a <font color = "red">significant and correlated</font> manner during a certain <font color = "red"> interval $[t_1, t_2]$ </font> of half-hours.
### Your task is to identify this interval $[t_1, t_2]$ as closely as possible, using <font color = "red">covariance analysis techniques</font>.
### Please use power method if you want to find the largest eigenvalue and its corresponding eigenvector for certain matrices.
### Please note that we are not interested in small, random noise.  We are looking for a large change in the covariance structure.